In [1]:
import os

import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
from sklearn.metrics import r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches 
import calendar

import datetime
plt.style.use('ggplot')

# Modeling concentration of PM2.5 across counties in California

__[Particulate matter (PM)](https://www.epa.gov/pm-pollution/particulate-matter-pm-basics)__ refers to tiny particles of solid or liquid suspended in the air. Particles with a diameter of 2.5 micrometers or smaller (PM2.5) are __[particularly concerning](https://ww2.arb.ca.gov/resources/inhalable-particulate-matter-and-health)__ because they can reach deep into the lungs and even the bloodstream. Exposure to PM2.5 has been linked to stroke, cardiovascular and respiratory disease, cancer, and other serious health outcomes. 

While daily concentrations above 35 μg/m³ are considered extremely hazardous, there is no known safe level of PM2.5 exposure.

PM2.5 comes from a wide range of sources, either emitted directly from vehicles, industrial activity, or fires, or formed in the atmosphere through chemical reactions involving pollutants like sulfur dioxide or nitrogen oxides. In California, wildfires are a major source. Ash and smoke contain __[high levels of PM2.5](https://www.epa.gov/wildfire-smoke-course/why-wildfire-smoke-health-concern)__, and wildfire events have become more frequent and intense with climate change.

In this notebook, we’ll build Bayesian hierarchical models to predict average monthly PM2.5 concentrations at the county level in California. We will use publicly available data provided by the U.S. __[Environmental Protection Agency (EPA)](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)__. Our goal is to account for spatial and temporal variation and eventually quantify uncertainty in these predictions.


### Load data

In [2]:
folder_path = ' '   # change this to your folder path
dataframes = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True).sort_values(by='Date')

## Exploratory data analysis

First, we’ll do a few quick checks: are there missing values, any NaNs, anything that looks unsual. If some data is missing, we’ll have to decide whether to drop or impute. We’ll also look at the distributions of the main variables because we need that to pick a reasonable likelihood for the Bayesian model.

In [ ]:
data.head()

In [ ]:
data.describe()

It seems like we have some negative value in `Daily Mean PM2.5 Concentration` column, which shouldn't happen. Let's inspect those values.

In [ ]:
plt.hist(data[data["Daily Mean PM2.5 Concentration"]<0]["Daily Mean PM2.5 Concentration"])
plt.show()

Negative values usually come from equipment malfunction, and we will in this tutorial replace them with zeros.

In [6]:
data.loc[data["Daily Mean PM2.5 Concentration"] < 0, "Daily Mean PM2.5 Concentration"] = 0

In [ ]:
data[data["Daily Mean PM2.5 Concentration"]<0]["Daily Mean PM2.5 Concentration"].sum() # the sum should we zero, we successfully replaced all negative values with zero

In [ ]:
data.isnull().any() # there are some NaNs in the dataset, but we're lucky, we don't have any NaNs columns we will use

We want to fit a model on data from 2019 to 2024, and treat everything we have in 2025 as 'unseen data'. First, we need to ensure we have data for all counties during that period, as monitoring stations can sometimes go offline, be decommissioned, or fail to collect data for certain stretches of time.

In [9]:
# drop data for 2025
data['Date'] = pd.to_datetime(data['Date'])

rows_to_drop = data[data['Date'] > datetime.datetime(2024, 12, 31)].index

data = data.drop(rows_to_drop)

In [ ]:
counties_list = data['County'].unique().tolist()

print("These counties have some data missing in the last 6 years:")

missing_counties = []
missing_dates_expanded = []
percentage_missing_list = [] 

# Define the expected date range for the last 6 years
start_date = "2019-01-01"
end_date = "2024-12-31"
expected_dates = pd.date_range(start=start_date, end=end_date, freq='D')

for county in counties_list:
    county_data = data[data['County'] == county]
    
    # Ensure only unique dates are considered
    actual_dates = pd.to_datetime(county_data['Date']).drop_duplicates().sort_values()
    
    # Find missing dates by comparing expected and actual dates
    missing_dates = expected_dates.difference(actual_dates)
    
    if len(missing_dates) > 0: 
        missing_counties.append(county)
        percentage_missing = np.round((100 - len(actual_dates) / len(expected_dates) * 100), 2)
        percentage_missing_list.append({"County": county, "Percentage Missing": percentage_missing})
        
        for date in missing_dates:
            missing_dates_expanded.append({"County": county, "Missing Date": date})
        print(county, "has", percentage_missing, "% of data missing")

missing_dates_df = pd.DataFrame(missing_dates_expanded)
percentage_missing_df = pd.DataFrame(percentage_missing_list)
missing_dates_df = missing_dates_df.merge(percentage_missing_df, on="County", how="left")

Most counties with missing dates have a very low percentage of missing data, which probably means there’s just an occasional missing date. In those cases, we can impute average values.

However, a few counties have a substantially higher proportion of missing dates, like Napa (61.82%) or Humboldt (53.6%). This likely indicates that monitoring stations went offline for an extended period. In such cases, it doesn’t make sense to impute average values. A common rule of thumb is that imputation is reasonable when less than 25% of the data is missing, although  this heavily depends on the context.

In our case, Shasta County has 25.55% missing data. That’s borderline, so we could reasonably go either way. I decided to keep it, since we're already excluding a few other counties.

In [ ]:
counties_to_drop = percentage_missing_df[percentage_missing_df["Percentage Missing"] >= 26]["County"].tolist()

data = data[~data["County"].isin(counties_to_drop)].reset_index(drop=True)

remaining_counties = data["County"].unique()
print(f"Remaining counties: {len(remaining_counties)}")
print(f"Counties still in data: {remaining_counties}")

### Impute data

In [12]:
# Filter out counties to drop
missing_dates_df = missing_dates_df[~missing_dates_df['County'].isin(counties_to_drop)].reset_index(drop=True)

new_rows = []

for county in missing_dates_df['County'].unique():
    county_missing_dates = missing_dates_df[missing_dates_df['County'] == county]['Missing Date']
    
    # Get a sample row for the county to copy other column values
    sample_row = data[data['County'] == county].iloc[0].to_dict()
    
    # Calculate mean values for specific columns
    mean_values = data[data['County'] == county][['Daily Mean PM2.5 Concentration', 'Daily AQI Value', 'Daily Obs Count']].mean()

    # Create new rows for missing dates
    for missing_date in county_missing_dates:
        new_row = sample_row.copy() 
        new_row['Date'] = missing_date 
        new_row.update(mean_values.to_dict())
        new_rows.append(new_row)

data = pd.concat([data, pd.DataFrame(new_rows)], ignore_index=True)
data = data.sort_values(by=['County', 'Date']).reset_index(drop=True)

In [ ]:
# Check if there are any missing dates in the entire data DataFrame
start_date = "2019-01-01"
end_date = "2024-12-31"
expected_dates = pd.date_range(start=start_date, end=end_date, freq='D')

actual_dates = pd.to_datetime(data['Date']).drop_duplicates().sort_values()
missing_dates = expected_dates.difference(actual_dates)

if len(missing_dates) > 0:
    print(f"The dataset still has {len(missing_dates)} missing dates.")
else:
    print("The dataset has no missing dates.")

### Time-series of daily PM2.5 concentration

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=data, x="Date", y="Daily Mean PM2.5 Concentration")

tick_step = 90 # put ticks every 3 months
unique_dates = pd.Series(data['Date'].unique()) 
plt.xticks(ticks=unique_dates[::tick_step], 
           labels=unique_dates[::tick_step].dt.strftime('%Y-%m'), 
           rotation=45)

plt.xlabel("Date")
plt.ylabel("PM2.5 Concentration")
plt.title("Daily Average PM2.5 Concentration in California (2019-2024)")
plt.show()

We see that on most days, daily concentration in California is around 20 μg/m³ or less. There are some big spikes in 2020 and 2021, which probably come from a big wildfire or similar extreme event.

### Does data have seasonality?

Before developing a model, we should check whether the data exhibits any seasonal patterns.

In [ ]:
data["month_name"] = data["Date"].dt.strftime('%b')
data["month_num"] = data["Date"].dt.month  # for correct ordering

# Group by month number to preserve order
monthly_seasonal_avg = data.groupby("month_num")["Daily Mean PM2.5 Concentration"].mean().reset_index()

monthly_seasonal_avg["month_name"] = monthly_seasonal_avg["month_num"].apply(lambda x: calendar.month_abbr[x]) 

sns.lineplot(x='month_name', y='Daily Mean PM2.5 Concentration', data=monthly_seasonal_avg, marker='o', color='teal')
plt.title("Average PM2.5 by Calendar Month")
plt.xlabel("Month")
plt.ylabel("Average PM2.5")
plt.tight_layout()
plt.show()


We see higher average monthly concentration of PM2.5 in September and toward the end of the year and in January. 

This pattern likely reflects a combination of factors: 

* wildfire activity in late summer and fall, driven in part by strong dry winds such as the __[Santa Ana winds](https://en.wikipedia.org/wiki/Santa_Ana_winds)__ in Southern California and __[Diablo winds](https://en.wikipedia.org/wiki/Diablo_wind)__ in Northern California, which rapidly spread fires and worsen air quality

* winter pollution events like __[temperature inversions](https://www.eea.europa.eu/en/analysis/maps-and-charts/temperature-inversion-traps-pollution-at)__ and residential heating contribute to elevated PM2.5 levels in the colder months.

## Calculate monthly average

We have daily data, but for the rest of the tutorial we’ll use monthly averages. Using monthly data reduces computational load and helps the model run faster. It also smooths out short-term fluctuations, which makes it easier to focus on longer-term trends.

We will now calculate monthly average concentration of PM2.5 and some variables that will help us incorproate seasonality into our model later.

In [16]:
# Sort the DataFrame by 'county' and 'year_month'and extract the first date for each county
# We know that first date everywhere should be 2019-01-01, 
# but let's do it automatically just in case one county has no data for that month 
data = data.sort_values(by=['County', 'Date'])

first_dates = data.groupby('County')['Date'].first().reset_index()
first_dates.rename(columns={'Date': 'first_date'}, inplace=True)  # Rename the column to 'first_date'

data = data.merge(first_dates, on='County', how='left')

In [17]:
# compute the total time since beginning of time-serials (in years)
# This will be later used to calculate seasonality and trend

data["time_passed_total"] = (
    data['Date'].apply(datetime.datetime.toordinal)
    -data['first_date'].apply(datetime.datetime.toordinal)
)/365.0

In [18]:
data_copy = data.copy()

data_copy['year_month'] = data_copy['Date'].dt.strftime('%Y-%m')

monthly_avg = data_copy.groupby(['County', 'year_month'])['Daily Mean PM2.5 Concentration'].mean().reset_index()

monthly_avg.rename(columns={'Daily Mean PM2.5 Concentration': 'Monthly Avg PM2.5'}, inplace=True)

first_dates = monthly_avg.groupby('County')['year_month'].first().reset_index()
first_dates.rename(columns={'year_month': 'first_date'}, inplace=True)

monthly_avg = monthly_avg.merge(first_dates, on='County', how='left')

monthly_avg['year_month'] = pd.to_datetime(monthly_avg['year_month'], format='%Y-%m')
monthly_avg['first_date'] = pd.to_datetime(monthly_avg['first_date'], format='%Y-%m')

# Calculate time passed in years since county's first month (for trend)
monthly_avg["time_passed_total"] = (
    monthly_avg['year_month'].apply(datetime.datetime.toordinal)
    - monthly_avg['first_date'].apply(datetime.datetime.toordinal)
) / 365.0


monthly_avg['month'] = monthly_avg['year_month'].dt.month

# Normalize month to [0,1) for seasonality
monthly_avg['month_fraction'] = (monthly_avg['month'] - 1) / 12.0

monthly_avg['county_id'] = monthly_avg['County'].astype('category').cat.codes
n_counties = monthly_avg['county_id'].nunique()

In [ ]:
# this is how our new dataframe looks like
monthly_avg.head()

### Plot distribution of monthly average

In [ ]:
plt.figure(figsize=(12, 6))

sns.histplot(monthly_avg["Monthly Avg PM2.5"], kde=True)

plt.title("Distribution of Monthly Average PM2.5 Concentration in California (2019-2024)")
plt.show()

As expected, many data points are between 0 and 20. There's also a small bump between 20 and 50, and a long tail. Let's plot a distribution for a couple of counties to get a better sense of data.

In [ ]:
counties = ['Los Angeles', 'Fresno', 'San Francisco', 'San Diego']


fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, county in zip(axes.flatten(), counties):
    county_data = monthly_avg[monthly_avg['County'] == county]
    sns.kdeplot(np.log(county_data['Monthly Avg PM2.5']), color='cornflowerblue', ax=ax)
    ax.set_title(county)
    ax.set_xlabel('Monthly Avg PM2.5')
    ax.set_ylabel('Density')

plt.tight_layout()
plt.show()

Looking at these distributions, it’s clear the data doesn’t follow a Normal distribution. It looks like it might be bimodal. It's probably a mix of two distributions, which would make sense. On most days PM2.5 is low, but during wildfires it can get very high.

It seems like on "good days" data follows a __[LogNormal distribution](https://en.wikipedia.org/wiki/Log-normal_distribution)__ (no negative values), but bad days it probably follows a different LogNormal with a higher mean. That pattern suggests there might be some kind of threshold or shift when extreme events occur.

We will now use this information to fit a Bayesian model.

## Fitting a Bayesian hierarchical model

### Bayesian hierarchical models

__[Bayesian hierarchical models](https://en.wikipedia.org/wiki/Bayesian_hierarchical_modeling)__ are a class of statistical models that involve multiple levels of parameters, allowing for the modeling of complex, structured data. They are particularly useful when data is organized in groups or layers, such as nested or repeated measurements, or when modeling different regions that share common characteristics. In these models, parameters at one level of the hierarchy depend on parameters at a higher level, facilitating the sharing of information across different groups. This approach leads to more accurate and robust inferences, especially when some groups, have limited data.

### Fitting a model in PyMC

We will build our models using __[PyMC](https://www.pymc.io/projects/docs/en/stable/learn.html)__, a python package for Bayesian inference. 

We’ll use PyMC, a Python library for building Bayesian models and running inference with MCMC. It’s useful when you want to define probabilistic models explicitly, use priors, and get full posterior distributions instead of point estimates. PyMC lets you write models in code that closely mirrors their mathematical structure, and it supports advanced features like hierarchical models, custom likelihoods, and missing data handling.

If you're starting with PyMC, I suggest reading about __[its core features](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/index.html)__ , the __[Introductiory Overview of PyMC](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html)__ or how to fit a __[GLM regression](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/GLM_linear.html)__ in PyMC.


In PyMC, models are defined inside a `pm.Model()` context manager. All components (priors, likelihoods, transformations, and observed data) must be declared within this block so PyMC can track them and handle inference properly.

Within the model context, we will define:

* **Priors** - distributions that reflect our assumptions before seeing the data,

* **The expected value and standard deviation** of the likelihood, and

* **The likelihood function** itself, which links parameters to the observed data.

To fit the model, we will use __[Markov Chain Monte Carlo (MCMC)](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo)__. Directly computing the posterior distribution is infeasible for most real-world models, so we approximate it using MCMC. This generates a sequence of samples (called *draws*) from the posterior distribution, which we can then use to estimate parameter values, uncertainty intervals, or make predictions.

Each **draw** is one sample from the posterior, and represents **one possible combination of values for all the model’s parameters at once**. This means that for every parameter in the model (e.g., intercepts, slopes, variances, or group-level effects), you get one specific value in that draw. 

So, when PyMC reports that "*50 draws are finished*" it means the sampler has generated 50 such samples and each contains values for all parameters. These 50 draws together represent 50 different possible sets of parameter values drawn from the posterior distribution.

A **chain** is an independent sequence of such draws. Running multiple chains (e.g. `chains=4`) helps us check whether sampling has converged (i.e., whether different chains explore the same distribution).

The output of `pm.sample()` is called a **trace**. It contains all the MCMC draws across all chains and parameters.

You can think of the trace as a structured container (like a multidimensional array or cube), which stores all sampled combinations of model parameters. In hierarchical models, it represents an entire posterior predictive distribution for each group or time series.

## Model 1: No hierarchy

To start, we’ll fit a non-hierarchical model. This simpler version will make it easier to explore PyMC’s features and be our a sanity check when we compare it to more complex models.

In this model, we include seasonality because the data clearly shows a strong seasonal pattern. We also define an intercept to capture the baseline level.

Since most days in our data don’t contain extreme values but occasional extreme events do occur, we model the probability of extremes using a __[Bernoulli distribution](https://www.pymc.io/projects/docs/en/stable/api/distributions/generated/pymc.Bernoulli.html)__. Depending on whether an extreme event occurs, the model switches between two __[LogNormal](https://www.pymc.io/projects/docs/en/stable/api/distributions/generated/pymc.LogNormal.html)__ likelihoods, one for typical days and one for extreme days.

In [ ]:
with pm.Model() as model_no_hierarchy:

    # --- Priors for seasonality ---
    # We model seasonal effects using sine and cosine terms to capture cyclical patterns over months.
    # The month_fraction variable represents the time within a year as a fraction (e.g., 0.0 to 1.0).
    # Taking sine and cosine of 2π * month_fraction captures annual seasonality.
    
    sine = pm.math.sin(2 * np.pi * (monthly_avg["month_fraction"].values % 1))
    cosine = pm.math.cos(2 * np.pi * (monthly_avg["month_fraction"].values % 1))

    # We assign Normal priors to the coefficients of the sine and cosine terms,
    # reflecting uncertainty about the amplitude of seasonal effects.
    w_s = pm.Normal("w_s", mu=0, sigma=1)
    w_c = pm.Normal("w_c", mu=0, sigma=1)

    # The overall seasonal effect is a linear combination of sine and cosine terms weighted by w_s and w_c.
    seasonality = sine * w_s + cosine * w_c

    # --- Priors for intercept ---
    # The intercept represents the baseline log-level of PM2.5 without seasonality or extremes.
    # We define a with a wide prior for the mean intercept (intercept_mu)
    # and a narrower offset term (intercept_offset) to allow some flexibility.

    intercept_mu = pm.Normal("intercept_mu", mu=0, sigma=5)  # wide prior on mean
    intercept_offset = pm.Normal("intercept_offset", mu=0, sigma=1)

    intercept = intercept_mu + intercept_offset

    # --- Modeling extreme events ---
    # We create a binary indicator variable 'is_extreme' to capture days with unusually high PM2.5.
    # The probability of an extreme event is modeled with a Beta prior to allow it to be low but flexible.
    
    p_is_extreme = pm.Beta("p_is_extreme", alpha=1, beta=20)
    is_extreme = pm.Bernoulli("is_extreme", p_is_extreme, shape=monthly_avg.shape[0])  

    # --- Expected value of PM2.5 ---
    # The baseline expected log PM2.5 is the sum of intercept and seasonal effect (mu_base).
   
    mu_base = intercept + seasonality

    # For extreme events, we allow a separate, higher mean value (mu_extreme) with a fairly wide prior.
    mu_extreme = pm.Normal("mu_extreme", mu=5, sigma=5)

    # We switch between baseline and extreme expected values depending on 'is_extreme'.
    mu = pm.math.switch(is_extreme, mu_extreme, mu_base)

    # --- Priors for standard deviations ---
    # We model measurement noise or natural variability with two HalfNormal priors:
    # one for the baseline condition (sigma_base), and one for extreme events (sigma_extreme).

    sigma_base = pm.HalfNormal("sigma_base", sigma=1)
    sigma_extreme = pm.HalfNormal("sigma_extreme", sigma=1)

    # We switch the standard deviation based on 'is_extreme' status.

    sigma = pm.math.switch(is_extreme, sigma_extreme, sigma_base)

    # --- Likelihood ---
    # The observed PM2.5 values are modeled as LogNormal distributed,
    # since PM2.5 concentrations are positive and often right-skewed.
    # The LogNormal distribution uses the 'mu' and 'sigma' defined above.
  
    Y = pm.LogNormal("Y", 
                    mu=mu, 
                    sigma=sigma, 
                    observed=monthly_avg['Monthly Avg PM2.5'])
    
    # --- Sampling ---
    # We draw 1000 posterior samples per chain after 500 tuning steps,
    # using 4 independent chains run in parallel on 4 CPU cores.

    trace_no_hierarchy = pm.sample(1000, tune=500, chains=4, cores=4, return_inferencedata=True)

    # --- Compute log likelihood ---
    # This is useful for model comparison or further diagnostics.
    pm.compute_log_likelihood(trace_no_hierarchy)


### Inspect trace object and visualize the model

In [ ]:
trace_no_hierarchy

In [ ]:
# Let's look at the posterior samples directly
trace_no_hierarchy.posterior

In [ ]:
# view the samples for 'intercept_mu'
trace_no_hierarchy.posterior["intercept_mu"]

In [ ]:
print("dimensions: ", trace_no_hierarchy.posterior["intercept_mu"].dims)
print("shape: ", trace_no_hierarchy.posterior["intercept_mu"].shape)

In [ ]:
# We can also print the model object to inspect its structure.
# This shows the model's components (priors, likelihoods, etc.) and how they are connected.
# It’s useful for verifying that everything is defined as expected.

model_no_hierarchy

In [ ]:
# We can also visualize the model structure using Graphviz.
# This generates a computational graph showing how all variables are connected in the model

pm.model_to_graphviz(model_no_hierarchy)

### Convergence checks

The R-hat statistic, also known as the __[Gelman-Rubin diagnostic](https://en.wikipedia.org/wiki/Gelman-Rubin_statistic)__, is used to assess the convergence of MCMC chains. It compares the variance within each chain to the variance between chains. If the chains have converged, the within-chain variance should be similar to the between-chain variance, and R-hat should be close to 1. An R-hat value significantly greater than 1.01 indicates that the chains have not yet converged, suggesting that more iterations may be needed to obtain reliable estimates from the MCMC sampling.

We will use `az.summary()` to get for an overview of the posterior distributions. 

For each parameter, the function shows:

* `mean`: the average of the posterior samples

* `sd`: standard deviation, showing uncertainty

* `hdi_3%` / `hdi_97%`: lower and upper bounds of the 94% highest density interval (by default), representing the range where most values fall

* `r_hat`: a convergence diagnostic (should be close to 1.0, or not higher than 1.01)

* `ess_bulk` / `ess_tail`: effective sample sizes, it measures how much independent information the samples contain

In [ ]:
az.summary(trace_no_hierarchy,
           var_names=['w_s', 
                      'w_c', 
                      'intercept_mu', 
                      'intercept_offset', 
                      'mu_extreme', 
                      'sigma_base', 
                      'sigma_extreme', 
                      'p_is_extreme'])


This looks good so far!

In [ ]:
az.summary(trace_no_hierarchy,
           var_names=['is_extreme'])


However, here we start to see issues. Although our model correctly estimates that `is_extreme` is close to 0 for most observations, diagnostics like effective sample size or R-hat show problems. Because we sample `is_extreme` for each observation, we introduce a large number of discrete latent variables (one per data point). 

Issues happens because:

* Gradient-based samplers like NUTS struggle with discrete variables,

* Sampling thousands of binary latent variables individually can lead to inefficient exploration and stuck chains.

For now, we accept this limitation and continue analyzing the model since we still get reasonable estimates for is_extreme. Fixing these issues would require more advanced steps like marginalizing out the discrete variables or using samplers designed for discrete parameters (project ideas?).


In [ ]:
az.plot_forest(trace_no_hierarchy, 
               var_names=['w_s', 
                          'w_c', 
                          'intercept_mu', 
                          'intercept_offset', 
                          'mu_extreme', 
                          'sigma_base', 
                          'sigma_extreme', 
                          'p_is_extreme'], 
               combined=True, r_hat=True)
plt.show()

We have a forest plot showing the posterior distributions and 94% credible intervals of selected parameters, with R-hat values to assess convergence across chains.

In [ ]:
az.plot_trace(trace_no_hierarchy, 
              var_names=['w_s', 
                         'w_c', 
                         'intercept_mu', 
                         'intercept_offset', 
                         'mu_extreme', 
                         'sigma_base', 
                         'sigma_extreme', 
                         'p_is_extreme'], 
              compact=True)

plt.tight_layout()

This figure displays the posterior distributions of parameters (left) and the corresponding autocorrelation plots (right) for parameters. The posterior distributions illustrate the range of parameter values after sampling. The autocorrelation plots on the right assess the efficiency and convergence of the MCMC sampling, where lower autocorrelation indicates more efficient sampling. **Different dashed line styles represent the different chains** used during sampling, helping to verify whether the chains have converged to a similar distribution.

## Model 2: County-hierarchy

We will now fit a Bayesian hierarchical model. As mentioned earlier, Bayesian models can include hierarchies of parameters. To start, we’ll add an intercept for each county to capture differences in their baseline PM2.5 level, because it’s expected that these baselines vary across counties.

In [ ]:
with pm.Model() as model_county_hierarchy:

    # --- Priors for seasonality ---
    sine = pm.math.sin(2 * np.pi * (monthly_avg["month_fraction"].values % 1))
    cosine = pm.math.cos(2 * np.pi * (monthly_avg["month_fraction"].values % 1))

    w_s = pm.Normal("w_s", mu=0, sigma=1)
    w_c = pm.Normal("w_c", mu=0, sigma=1)

    seasonality = sine * w_s + cosine * w_c

    # --- Priors for intercept ---


    # --- Modeling extreme events ---
    # We create a binary indicator variable 'is_extreme' to capture days with unusually high PM2.5.
    # The probability of an extreme event is modeled with a Beta prior to allow it to be low but flexible.
    
    p_is_extreme = pm.Beta("p_is_extreme", alpha=1, beta=20)
    is_extreme = pm.Bernoulli("is_extreme", p_is_extreme, shape=monthly_avg.shape[0])  

    # --- Expected value of PM2.5 ---
    # The baseline expected log PM2.5 is the sum of intercept and seasonal effect (mu_base).
   
    mu_base = intercept + seasonality

    # For extreme events, we allow a separate, higher mean value (mu_extreme) with a fairly wide prior.
    mu_extreme = pm.Normal("mu_extreme", mu=5, sigma=5)

    # We switch between baseline and extreme expected values depending on 'is_extreme'.
    mu = pm.math.switch(is_extreme, mu_extreme, mu_base)

    # --- Priors for standard deviations ---
    # We model measurement noise or natural variability with two HalfNormal priors:
    # one for the baseline condition (sigma_base), and one for extreme events (sigma_extreme).

    sigma_base = pm.HalfNormal("sigma_base", sigma=1)
    sigma_extreme = pm.HalfNormal("sigma_extreme", sigma=1)

    # We switch the standard deviation based on 'is_extreme' status.

    sigma = pm.math.switch(is_extreme, sigma_extreme, sigma_base)

    # --- Likelihood ---
    Y = pm.LogNormal("Y", 
                    mu=mu, 
                    sigma=sigma, 
                    observed=monthly_avg['Monthly Avg PM2.5'])

    trace_county_hierarchy = pm.sample(1000, tune=500, chains=4, cores=4, return_inferencedata=True)

    pm.compute_log_likelihood(trace_county_hierarchy)


In [ ]:
# print the model formula


In [ ]:
# visualize the model structure using Graphviz



In [ ]:
az.summary(trace_county_hierarchy,
           var_names=['w_s', 
                      'w_c', 
                      'intercept_mu', 
                      'intercept_offset', 
                      'mu_extreme', 
                      'sigma_base', 
                      'sigma_extreme', 
                      'p_is_extreme'])


We see that some `r_hat` values are above 1.01, which suggests convergence issues. This can sometimes be improved by increasing the number of tuning steps, samples, or chains.

However, high `r_hat` might also suggest deeper issues with the model, like poor parameterization or parameters that aren’t well identified.

In [ ]:
# plot credible intervals for posterior distributions and r_hat using plot_forest function


In [ ]:
az.plot_trace(trace_county_hierarchy, 
              var_names=['w_s', 
                         'w_c', 
                         'intercept_mu', 
                         'intercept_offset', 
                         'mu_extreme', 
                         'sigma_base', 
                         'sigma_extreme', 
                         'p_is_extreme'], 
              compact=True)

plt.tight_layout()

## Model 3: County- and trend-hierarchy 

Since our model with county-specific intercepts still has some issues, we’ll now expand it by adding a trend component.

So far, we haven’t explicitly modeled any long-term trend. But interestingly, PM2.5 levels across the US have been __[declining overall](https://www.epa.gov/air-trends/particulate-matter-pm25-trends)__, while in California, the number and intensity of wildfires has increased (see __[here](https://emlab.ucsb.edu/sites/default/files/documents/wildfire-brief.pdf)__ and __[here](https://climateresilience.ca.gov/overview/impacts.html)__). That means while levels of PM2.5 are dropping on average, extreme spikes from wildfires are becoming more common.

To capture this, we’ll now give each county its own trend line over time.

In [ ]:
with pm.Model() as model_county_trend_hierarchy:

    # --- Priors for seasonality ---
    sine = pm.math.sin(2 * np.pi * (monthly_avg["month_fraction"].values % 1))
    cosine = pm.math.cos(2 * np.pi * (monthly_avg["month_fraction"].values % 1))

    w_s = pm.Normal("w_s", mu=0, sigma=1)
    w_c = pm.Normal("w_c", mu=0, sigma=1)

    seasonality = sine * w_s + cosine * w_c

    # --- Priors for intercept ---
    intercept_mu = pm.Normal("intercept_mu", mu=0, sigma=5)  # wide prior on mean
    
    intercept_offset = pm.Normal("intercept_offset", mu=0, sigma=1, shape=n_counties)
    intercept_raw = intercept_mu + intercept_offset

    # intercept hierachy
    intercept = intercept_raw[monthly_avg['county_id']]

    # --- Priors for trend ---
    
    # Global average trend across all counties (centered at 0, wide prior)
    mu_trend = 
    
    # County-specific deviations from the global trend (one per county)
    trend_offset = 
    
    # Combine global and county-specific components to get per-county trend slopes
    weight_trend = mu_trend + trend_offset
    
    # Match each observation to its county's trend slope and multiply by time
    trend = 

    # --- Modeling extreme events ---
    # We create a binary indicator variable 'is_extreme' to capture days with unusually high PM2.5.
    # The probability of an extreme event is modeled with a Beta prior to allow it to be low but flexible.
    
    p_is_extreme = pm.Beta("p_is_extreme", alpha=1, beta=50) # very narrow prior
    is_extreme = pm.Bernoulli("is_extreme", p_is_extreme, shape=monthly_avg.shape[0])  

    # --- Expected value of PM2.5 ---
    # The baseline expected log PM2.5 is the sum of intercept and seasonal effect (mu_base).
   
    mu_base = intercept + seasonality

    # For extreme events, we allow a separate, higher mean value (mu_extreme) with a fairly wide prior.
    mu_extreme = pm.Normal("mu_extreme", mu=5, sigma=5)

    # We switch between baseline and extreme expected values depending on 'is_extreme'.
    mu = pm.math.switch(is_extreme, mu_extreme, mu_base)

    # --- Priors for standard deviations ---
    # We model measurement noise or natural variability with two HalfNormal priors:
    # one for the baseline condition (sigma_base), and one for extreme events (sigma_extreme).

    sigma_base = pm.HalfNormal("sigma_base", sigma=1)
    sigma_extreme = pm.HalfNormal("sigma_extreme", sigma=1)

    # We switch the standard deviation based on 'is_extreme' status.

    sigma = pm.math.switch(is_extreme, sigma_extreme, sigma_base)

    # --- Likelihood ---
    Y = pm.LogNormal("Y", 
                    mu=mu, 
                    sigma=sigma, 
                    observed=monthly_avg['Monthly Avg PM2.5'])

    trace_county_trend_hierarchy = pm.sample(1000, tune=500, chains=4, cores=4, return_inferencedata=True)

    pm.compute_log_likelihood(trace_county_trend_hierarchy)


In [ ]:
model_county_trend_hierarchy

In [ ]:
pm.model_to_graphviz(model_county_trend_hierarchy)

In [ ]:
az.summary(trace_county_trend_hierarchy, 
           var_names=['w_s', 
                      'w_c', 
                      'intercept_mu', 
                      'intercept_offset', 
                      'mu_extreme', 
                      'sigma_base', 
                      'sigma_extreme', 
                      'p_is_extreme'])


In [ ]:
az.summary(trace_county_trend_hierarchy, 
           var_names=['mu_trend',
                      'trend_offset']) 

This looks better than the model with only county intercepts. `r_hat` is still above 1.01 for some parameters, but that might go away if we increase the number of tuning steps or samples.

In [ ]:
az.plot_forest(trace_county_trend_hierarchy, 
               var_names=['w_s', 
                          'w_c', 
                          'intercept_mu', 
                          'intercept_offset', 
                          'mu_extreme', 
                          'sigma_base', 
                          'sigma_extreme', 
                          'p_is_extreme', 
                          'mu_trend',
                          'trend_offset'], 
               combined=True, 
               r_hat=True)

plt.show()

In [ ]:
az.plot_trace(trace_county_trend_hierarchy, 
              var_names=['w_s', 
                          'w_c', 
                          'intercept_mu', 
                          'intercept_offset', 
                          'mu_extreme', 
                          'sigma_base', 
                          'sigma_extreme', 
                          'p_is_extreme', 
                          'mu_trend',
                          'trend_offset'], 
              compact=True)

plt.tight_layout()

## Generating predictions based on posterior distribution of the model

We now have 3 models of different complexity. So far we’ve looked at convergence diagnostics, but we haven’t yet compared their predictions.

In [ ]:
models = {
    "model_no_hierarchy": {"model": model_no_hierarchy, "trace": trace_no_hierarchy},
    "model_county_hierarchy": {"model": model_county_hierarchy, "trace": trace_county_hierarchy},
    "model_county_trend_hierarchy": {"model": model_county_trend_hierarchy, "trace": trace_county_trend_hierarchy},
}

for label, items in models.items():
    ppc = pm.sample_posterior_predictive(items["trace"], model=items["model"], var_names=["Y"])
    items["ppc"] = ppc.posterior_predictive["Y"].values

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharey=True)

for ax, (label, items) in zip(axes, models.items()):
    samples = items["ppc"].flatten()
    sns.kdeplot(samples, ax=ax, label="Posterior Predictive", fill=True, alpha=0.6)
    sns.kdeplot(monthly_avg["Monthly Avg PM2.5"], ax=ax, label="Observed Data", color="black", linestyle="--")
    
    ax.set_title(label)
    ax.set_xlabel("Monthly Avg PM2.5")
    ax.set_ylabel("Density")
    ax.legend()

axes[2].set_xlabel("Monthly Avg PM2.5") 
plt.suptitle("Posterior Predictive Check (KDE) per Model")
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


This figure visually compares the distribution of model-predicted values (orange shade) with the actual observed data (dashed black line). The comparison is based on predictions derived from the model's posterior distribution, checked against the observed data to assess consistency. The misalignment between the two distributions at their peaks and tails suggests that the model may have difficulty accurately predicting certain data points, i.e. outliers.

### Compute predictions

We will compute the mean of the posterior predictive samples to obtain predictions and calculate percentiles from these samples to represent prediction uncertainty. Although the Bayesian approach also allows for determining credible intervals for model parameters to assess their uncertainty, I will focus on using percentiles of the predictions for easier data visualization.

In [48]:
for label, items in models.items():
    ppc_samples = items["ppc"]  # posterior predictive samples: shape (chains, draws, obs)
    trace = items["trace"]
    
    # Average over chains and draws to get predicted means for each observation
    predicted_means = ppc_samples.mean(axis=0).mean(axis=0)
    items["predicted_means"] = predicted_means
    
    # Mean of is_extreme across chains and draws for each observation
    is_extreme_mean = trace.posterior["is_extreme"].mean(dim=["chain", "draw"]).values
    items["is_extreme_mean"] = is_extreme_mean
    
    # Calculate 2.5% and 97.5% quantiles for credible intervals
    lower_bound = np.percentile(ppc_samples.mean(axis=0), 2.5, axis=0)
    upper_bound = np.percentile(ppc_samples.mean(axis=0), 97.5, axis=0)
    items["lower_bound"] = lower_bound
    items["upper_bound"] = upper_bound


In [49]:
model_names = ['model_no_hierarchy', 'model_county_hierarchy', 'model_county_trend_hierarchy']
columns = ['predicted_means', 'lower_bound', 'upper_bound', 'is_extreme_mean']

for model in model_names:
    for col in columns:
        monthly_avg[f'{col}_{model.replace("model_", "")}'] = models[model][col]


### Calculate r2 scores

In [50]:
r2_results = {}

# Get unique counties and their names once
county_map = monthly_avg.drop_duplicates(subset=['county_id'])[['county_id', 'County']].set_index('county_id')['County']

county_ids = monthly_avg['county_id'].unique()

for model_label, items in models.items():
    r2_scores = []
    preds = items['predicted_means']  # predicted means array, length = number of observations
    
    for county_id in county_ids:
        mask = (monthly_avg['county_id'] == county_id)
        y_obs = monthly_avg.loc[mask, 'Monthly Avg PM2.5'].values
        y_pred = preds[mask.values]  # mask.values to align boolean index with numpy array
        
        r2 = r2_score(y_obs, y_pred)
        r2_scores.append(r2)
    
    r2_results[model_label] = r2_scores
    
    county_to_r2 = dict(zip(county_ids, r2_scores))
    
    monthly_avg[f'r2_{model_label}'] = monthly_avg['county_id'].map(county_to_r2)


In [ ]:
monthly_avg[monthly_avg["County"]=="Los Angeles"]

### Observed vs Predicted plot for each county

In [ ]:
plot_counties = ['Los Angeles', 'Fresno', 'San Francisco', 'San Diego']
filtered_data = monthly_avg[monthly_avg['County'].isin(plot_counties)]

model_to_column = {
    "No Hierarchy": "predicted_means_no_hierarchy",
    "County Hierarchy": "predicted_means_county_hierarchy",
    "County + Trend Hierarchy": "predicted_means_county_trend_hierarchy",
}

model_to_r2_col = {
    "No Hierarchy": "r2_model_no_hierarchy",
    "County Hierarchy": "r2_model_county_hierarchy",
    "County + Trend Hierarchy": "r2_model_county_trend_hierarchy",
}

colors = {
    "No Hierarchy": "blue",
    "County Hierarchy": "green",
    "County + Trend Hierarchy": "orange",
}

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for ax, county in zip(axes, plot_counties):
    county_data = filtered_data[filtered_data["County"] == county]

    min_val = min(county_data['Monthly Avg PM2.5'].min(), *(county_data[col].min() for col in model_to_column.values()))
    max_val = max(county_data['Monthly Avg PM2.5'].max(), *(county_data[col].max() for col in model_to_column.values()))
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction')

    for model_label, pred_col in model_to_column.items():
        r2_val = county_data[model_to_r2_col[model_label]].iloc[0]
        ax.scatter(
            county_data["Monthly Avg PM2.5"],
            county_data[pred_col],
            label=f"{model_label} (R² = {r2_val:.2f})",
            color=colors[model_label],
            alpha=0.6,
            s=35,
        )

    ax.set_title(county, fontsize=16)
    ax.set_xlabel("Observed PM2.5", fontsize=14)
    ax.set_ylabel("Predicted PM2.5", fontsize=14)
    ax.legend(fontsize=11)
    ax.tick_params(labelsize=12)

plt.tight_layout()
plt.show()


This plot compares the observed data (x-axis) with the predicted values (y-axis) generated by 3 model for 4 different cities. The diagonal red dashed line represents the line of perfect prediction. Points close to this line indicate better predictions, while scattered points indicate more significant discrepancies between the model’s predictions and the observed data.

### Predictions for each county

In [54]:
# helper function for plotting

def plot_model_predictions(data, model_col, lower_col, upper_col, is_extreme_col, plot_counties, title_suffix):
    filtered_data = data[data['County'].isin(plot_counties)].copy()
    filtered_data = filtered_data.rename(columns={
        model_col: 'predicted_means',
        lower_col: 'lower_bound',
        upper_col: 'upper_bound',
        is_extreme_col: 'is_extreme',
    })

    g = sns.FacetGrid(filtered_data, col='County', col_wrap=2, height=5, aspect=1.5, sharex=False, sharey=False)
    
    # Disable automatic legend in these plots:
    g.map(sns.scatterplot, 'year_month', 'Monthly Avg PM2.5', label='observations', color='black', alpha=0.7, legend=False)
    g.map(sns.lineplot, 'year_month', 'predicted_means', label='predictions', color='cornflowerblue', alpha=0.7, legend=False)

    fill_patch = patches.Patch(color='lightblue', alpha=0.5, label='95% Percentile')

    legend_handles = []
    legend_labels = []

    for ax, (region, subset) in zip(g.axes.flat, filtered_data.groupby('County')):
        ax.fill_between(subset['year_month'], 
                        subset['lower_bound'],
                        subset['upper_bound'], 
                        color='lightblue', alpha=0.5)

        ax2 = ax.twinx()
        ax2.set_ylabel('P(is extreme)', fontsize=12)
        ax2.set_ylim(0, 1)
        
        # Disable automatic legend here too:
        sns.lineplot(data=subset, x='year_month', y='is_extreme', ax=ax2, color='red', alpha=0.7, label='P(is extreme)', legend=False)
        
        if ax == g.axes.flat[0]:
            h1, l1 = ax.get_legend_handles_labels()
            h2, l2 = ax2.get_legend_handles_labels()
            legend_handles.extend(h1)
            legend_labels.extend(l1)
            legend_handles.extend(h2)
            legend_labels.extend(l2)

    legend_handles.append(fill_patch)
    legend_labels.append('95% Percentile')

    for ax, title in zip(g.axes.flat, g.col_names):
        ax.set_title(title, fontsize=16)

    for i, ax in enumerate(g.axes.flat):
        x_label = 'year_month' if i in [2, 3] else ''
        y_label = 'Monthly Avg PM2.5' if i in [0, 2] else ''
        ax.set_xlabel(x_label, fontsize=16 if x_label else None)
        ax.set_ylabel(y_label, fontsize=16 if y_label else None)
        ax.tick_params(axis='both', which='major', labelsize=12, length=6, width=1)
        ax.tick_params(axis='x', rotation=45)

    first_ax = g.axes.flat[0]
    first_ax.legend(legend_handles, legend_labels, fontsize=14, loc='upper right')

    plt.subplots_adjust(top=0.9)
    fig = g.figure
    fig.suptitle(f"Model Predictions: {title_suffix}", fontsize=20)

    plt.tight_layout()
    plt.show()


In [ ]:
plot_model_predictions(
    data=monthly_avg,
    model_col='predicted_means_no_hierarchy',
    lower_col='lower_bound_no_hierarchy',
    upper_col='upper_bound_no_hierarchy',
    is_extreme_col='is_extreme_mean_no_hierarchy',
    plot_counties=plot_counties,
    title_suffix='No Hierarchy'
)

plot_model_predictions(
    data=monthly_avg,
    model_col='predicted_means_county_hierarchy',
    lower_col='lower_bound_county_hierarchy',
    upper_col='upper_bound_county_hierarchy',
    is_extreme_col='is_extreme_mean_county_hierarchy',
    plot_counties=plot_counties,
    title_suffix='County Hierarchy'
)

plot_model_predictions(
    data=monthly_avg,
    model_col='predicted_means_county_trend_hierarchy',
    lower_col='lower_bound_county_trend_hierarchy',
    upper_col='upper_bound_county_trend_hierarchy',
    is_extreme_col='is_extreme_mean_county_trend_hierarchy',
    plot_counties=plot_counties,
    title_suffix='County + Trend Hierarchy'
)


Observed data (black dots) and predictions (blue lines) are shown for 4 counties. The shaded areas represent the 5%–95% prediction intervals (the uncertainty rang). Both hierarchical models fit the data somewhat better than the model without hierarchy. The `is_extreme` variable highlights unusual observations, though there is still room for improvement (more on this later).

# Model comparison

Let’s compare our three models using `az.compare()`. This shows how well each one generalizes by estimating out-of-sample predictive performance. 

We should pay attention to these metrics:

* **loo (Leave-One-Out cross-validation)**: Lower is better because it means the model predicts new data more accurately.

* **p_loo (Effective number of parameters)**: This tells us about model complexity. Higher values mean the model is more complex.

* **weight**: This gives the relative chance that a model is the best among the ones compared, based on stacking or averaging.

In [ ]:
compare_df = az.compare({"model_no_hierarchy": trace_no_hierarchy, 
                         "model_county_hierarchy": trace_county_hierarchy, 
                         "model_county_trend_hierarchy": trace_county_trend_hierarchy}, 
                         ic='loo', 
                         method = 'stacking')
display(compare_df)


In [ ]:
az.plot_compare(compare_df)
plt.show()

We can see that both hierarchical models clearly outperform the one without hierarchy. There’s a slight preference for the model that includes both county and trend hierarchy.

# Conclusion and further steps

This tutorial was an example of building Bayesian hierarchical models in PyMC, model inspection and model comparison. The model picks up on differences between counties and detects extreme events, it could still be improved.

Some ways to improve the model:

1. Add weather data such as average, minimum, and maximum temperature and humidity to better explain PM2.5 variation.

2. Include wildfire data, since PM2.5 spikes are often wildfire-related, though other pollution sources exist.

3. Investigate clusters in the data and consider adding a cluster-level hierarchy. For example, coastal vs. inland California differ in climate and pollution patterns, though urbanization might affect this relationship.

4. Experiment with the trend structure. Currently, each county has its own trend,  but a global trend might be enough.

5. Revisit priors and their parameters. The current choices worked well for the dataset but should be reevaluated as you add more data.

6. Examine issues with the `is_extreme` variable and its detection capabilities.

7. Consider using daily data instead of monthly averages. This would require modeling daily seasonality and accounting for weekday/holiday effects, but would increase model complexity and runtime.

6. Add your own ideas :)

After developing a model using data from 2019 to 2024, it would be good to test its predictive performance on 2025 data (unseen during training). Keep in mind that data availability for 2025 may vary across counties.

Data Sources: 
* https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/time-series
* https://catalog.data.gov/dataset?tags=wildfire
* https://projects.capradio.org/california-fire-history/
* https://scholars.georgiasouthern.edu/en/datasets/california-weather-and-fire-prediction-dataset-19842025-with-engi
* https://data.ca.gov/dataset/cal-fire
* https://www.kaggle.com/datasets/vivekattri/california-wildfire-damage-2014-feb2025
* https://zenodo.org/records/14712845

Keep in mind that some of these datasets are on the state or national level, some are on the county-level.


Links to some interesting papers about Bayesian modeling and PM2.5 I found (no need to implement these models, they're just a good starting point for further reading and inspiration):

* https://www.nature.com/articles/s41370-022-00471-4
* https://pmc.ncbi.nlm.nih.gov/articles/PMC8102649/
* https://onlinelibrary.wiley.com/doi/10.1155/2020/7135142
* https://ehp.niehs.nih.gov/doi/10.1289/ehp.6980
* https://doi.org/10.1371/journal.pone.0272774
* https://link.springer.com/article/10.1007/s11869-025-01762-z
* https://www.sciencedirect.com/science/article/pii/S0160412018303891
* https://ajehe.umsha.ac.ir/Article/58031
* https://www.mdpi.com/2073-4433/15/5/594
